# Data Cleaning

## 1. Introduccion

## 2. Impotando Librerias

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

## 3. Creacion de `df` utilizando datos de `CoreCode` en `data_core/`

### 3.1. Preparacion dataset `confirmed_global.csv`

In [ ]:
#url_confirmed_global = "https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv&filename=time_series_covid19_confirmed_global.csv"
#df1 = pd.read_csv(url_confirmed_global)
df1 = pd.read_csv('data_core/confirmed_global.csv')

In [ ]:
# El analisis se va a hacer por pais, no por provincia de modo que elimino la columna 'Province/State'. Las columnas de 'Lat' y 'Long' 
# se van a eliminar ahora para luego mergearlas con el dataframe final, ya que las coordenadas se cerian alteradas en el 'groupby'.

df1 = df1.drop(['Province/State'], axis=1)
df1 = df1.drop(['Lat'], axis=1)
df1 = df1.drop(['Long'], axis=1)

In [ ]:
# Una vez eliminada dichas columnas agrupamos los datos a nivel de fila por pais Sumando asi todos 
# los casos por pais que anteriormente estaban subdivididos por 'Province/State'.

# Comprobamos que efectivamente, hay nombres de paises que aparecen varias veces
print(df1["Country/Region"].value_counts().to_string())

In [ ]:
df1.loc[df1["Country/Region"] == "Austria"]

In [ ]:
# Vemos que tras el groupby los casos de agrupado correctamente, ya que la suma de la columna de casos de un dia especifico
# es igual a la fila de ese mismo dia para df1 tras esta operacion
df1 = df1.groupby(['Country/Region']).sum().reset_index()
print(df1.loc[df1["Country/Region"] == "Austria"].sum())

In [ ]:
# Vemos que solo existe un valor por pais. 
print(df1["Country/Region"].value_counts().to_string())

<div align="center">
Confirmamos que el groupby se ha completado con exito
<div>

In [ ]:
# Mergeamos las columnas de 'Date-Countrty' por cada pais y anadimos una columna con su valor correspondiente

# Agrupo las columnas de fecha en filas utilizando la funcion `melt` y hago un idetificador unico para mergear con el resto
# de tablas, que sera el (dia)+(el nombre del pais) para poder mergear correctamente con el resto de tablas por dia y pais
df1 = df1.melt(id_vars=["Country/Region"], 
        var_name="Date", 
        value_name="Confirmed")

# Creo la columna con el identificador para usarla como indentificador unico para el mergeo
df1['Date-Country'] = df1['Date'] + df1['Country/Region']

# Hago esta misma columna indice del dataframe
df1.set_index('Date-Country')

### 3.2. Preparacion dataset `deaths_global.csv`

Repetimos el mismo proceso anterior para el dataset `deaths_global.csv`

In [ ]:
#url_deaths_global = "https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_deaths_global.csv&filename=time_series_covid19_deaths_global.csv"
#df2 = pd.read_csv(url_deaths_global)
df2 = pd.read_csv('data_core/deaths_global.csv')

df2 = df2.drop(['Province/State'], axis=1)
df2 = df2.drop(['Lat'], axis=1)
df2 = df2.drop(['Long'], axis=1)
df2 = df2.groupby(['Country/Region']).sum().reset_index()
df2 = df2.melt(id_vars=["Country/Region"], 
        var_name="Date", 
        value_name="Deaths")
df2['Date-Country'] = df2['Date'] + df2['Country/Region']

df2.set_index('Date-Country')

### 3.3. Juntamos todos los dataframe `df1` y `df2` en uno solo `df`

In [ ]:
# Creo un primer dataframe final (df_f1), mergeando df1 y df2 por 'Date-Country'
df = pd.merge(df1, df2 , how='left', on='Date-Country')

In [ ]:
#Elimino las columnas duplicadas
df = df.drop(['Date-Country','Country/Region_y','Date_y'], axis=1)

# Reordeno las Columnas
df = df.rename(columns={'Country/Region_x':'country', 'Date_x':'date','Confirmed':'totalConfirmed','Deaths':'totalDeaths'})
df = df[['country','date','totalConfirmed','totalDeaths']]

In [ ]:
df_DD = df.drop_duplicates()

print(f"Filas df: {df.shape[0]}\nFilas df sin duplicados: {df_DD.shape[0]}")
n_duplicados = df.shape[0] - df_DD.shape[0]
print(f"Hay {n_duplicados} filas duplicadas")

In [ ]:
df['date'] = pd.to_datetime(df.date)
df = df.sort_values(['country','date'], ascending=[True, True])
df = df.reset_index(drop=True)

In [ ]:
df['confirmedDay'] = df['totalConfirmed'].diff().fillna(0).astype(int)
df['deathsDay'] = df['totalDeaths'].diff().fillna(0).astype(int)
df

In [ ]:
df.dtypes

## 4. Anado datos geograficos y poblacion a `df`

### 4.1. Importancion de datos 

In [ ]:
df4 = pd.read_csv("data_extra/concap.csv")

### 4.2. Preaparacion del dataframe

In [ ]:
df4 = df4.rename(columns={'CountryName':'country',
                          'CapitalLatitude':'latitude', 
                          'CapitalLongitude':'longitude', 
                          'CountryCode':'geoId',
                          'ContinentName':'continentExp'})
df4

In [ ]:
df_DD = df4.drop_duplicates()

print(f"Filas df4: {df4.shape[0]}\nFilas df4 sin duplicados: {df_DD.shape[0]}")

n_duplicados = df4.shape[0] - df_DD.shape[0]
print(f"Hay {n_duplicados} filas duplicadas")

df4 = df4.dropna()
n_null = df4.isnull().sum().sum()
print(f"Hay {n_null} registros nulos en total")

Control de calidad OK

In [ ]:
df = pd.merge(df, df4 , how='left', on='country')
df

In [ ]:
df_DD = df.drop_duplicates()

print(f"Filas df4: {df.shape[0]}\nFilas df4 sin duplicados: {df_DD.shape[0]}")

n_duplicados = df.shape[0] - df_DD.shape[0]
print(f"Hay {n_duplicados} filas duplicadas")

df = df.dropna()
n_null = df.isnull().sum().sum()
print(f"Hay {n_null} registros nulos en total")

Control de calidad ok

### 4.3. Filtro `df` por paises de `Europa`

In [ ]:
filter_europe = df['continentExp'] == 'Europe'
df = df[filter_europe]

In [ ]:
df = df.drop(['continentExp'], axis=1)
df

Dataframe prepardo para mergearlo y ser enriquecido

## 5. Union del dataset: `owid-covid-data.csv` con `df`

### 5.1. Creacion de `id` en `df` para el mergeo

In [ ]:
# Defino la columna que me serviran para mergear con otros dataset
df['id-merge'] = df['country'] + df['date'].apply(str)
df['id-merge'] = df['id-merge'].apply(lambda x: x.split(' ')[0])
df['id-merge']

In [ ]:
df_DD = df.drop_duplicates()

print(f"Filas df: {df.shape[0]}\nFilas df sin duplicados: {df_DD.shape[0]}")
n_duplicados = df.shape[0] - df_DD.shape[0]
print(f"Hay {n_duplicados} filas duplicadas")
n_null = df.isnull().sum().sum()
print(f"Hay {n_null} registros nulos en total")

### 5.1. Preaparcion del dataset `owid-covid-data.csv`

#### 5.1.1. Importacion de datos

In [ ]:
#url_confirmed_global = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
#df_ex = pd.read_csv(url_confirmed_global)
df_ex = pd.read_csv('data_extra/owid-covid-data.csv')

In [ ]:
df_ex.dtypes

#### 5.1.2. Tratamiento de las columnas

In [ ]:
df_ex.columns

In [ ]:
# Elijo las columnas que son relevantes
df_ex = df_ex.drop(df_ex.columns.difference(['continent','location', 'date', 'icu_patients','hosp_patients',
                                          'total_tests','positive_rate','tests_per_case',
                                          'new_vaccinations','people_vaccinated_per_hundred',
                                          'people_fully_vaccinated_per_hundred','population']), axis=1)


In [ ]:
df_ex = df_ex.rename(columns={'icu_patients':'icuPatients',
                          'hosp_patients':'hospPatients', 
                          'total_tests':'totalTests', 
                          'positive_rate':'positiveRate',
                          'tests_per_case':'testsPerCase',
                          'new_vaccinations':'newVaccinations',
                          'people_vaccinated_per_hundred':'vaccinatedPerHundred', 
                          'people_fully_vaccinated_per_hundred':'fullyVaccinatedPerHundred',
                          'location':'country'})

#### 5.1.3. Filtro el dataframe por paises `Europeos`

In [ ]:
#filter_europe = df_ex['continent'] == 'Europe'
#df_ex = df_ex[filter_europe]

# Elimino la columna de 'continent'
df_ex = df_ex.drop(['continent'], axis=1)

In [ ]:
df_ex['date'] = pd.to_datetime(df_ex.date)
df_ex = df_ex.sort_values(['country','date'], ascending=[True, True])
df_ex = df_ex.reset_index(drop=True)
df_ex

#### 5.1.4. Control de Calidad

In [ ]:
df_DD = df_ex.drop_duplicates()

print(f"Filas df: {df_ex.shape[0]}\nFilas df sin duplicados: {df_DD.shape[0]}")
n_duplicados = df_ex.shape[0] - df_DD.shape[0]
print(f"Hay {n_duplicados} filas duplicadas")
n_null = df_ex.isnull().sum().sum()
print(f"Hay {n_null} registros nulos en total")
print(df_ex.dtypes)

Vemos que hay un grancantidad de registros nulos, que mercen implementar acciones para eliminarlos.

#### 5.1.5. Relleno de Registros Nulos

**Signifcado de las columnas del dataset**
- **icu_patients**: Número de pacientes con COVID-19 en unidades de cuidados 
intensivos (UCI) en un día determinado

- **hosp_patients**: Número de pacientes con COVID-19 en el hospital en un día determinado

- **total_tests**: Pruebas totales para COVID-19

- **positive_rate**: La proporción de pruebas de COVID-19 que son positivas, expresada como un 
promedio móvil de 7 días (esto es lo contrario de las pruebas por caso)

- **tests_per_case**: Pruebas realizadas por cada nuevo caso confirmado de COVID-19, dado como un 
promedio móvil de 7 días (esto es lo contrario de Positive_rate)

- **new_vaccinations**: Nuevas dosis de vacuna COVID-19 administradas 
(solo calculadas para días consecutivos)

- **people_vaccinated_per_hundred**: Número total de personas que recibieron al menos una dosis 
de vacuna por cada 100 personas en la población total

- **people_fully_vaccinated_per_hundred**: Número total de personas que recibieron todas las dosis prescritas por el 
protocolo de vacunación por cada 100 personas en la población total

- **population**: poblacion total del pais

##### 5.1.5.1 Columnas `IcuPatients`,  `hospPatients`, `positive_rate`, `tests_per_case` y `new_vaccinations`

Al ser valores No continuos y por dia, es decir que los registros de cada linea es aislado del resto e indivisual, los registros nulos debemos de rellenarlos con `0` puesto que no tenmos mas informacion. 

In [ ]:
# Relleno valores nulos con 0
df_ex['icuPatients'] = df_ex['icuPatients'].fillna(0)
df_ex['hospPatients'] = df_ex['hospPatients'].fillna(0)
df_ex['positiveRate'] = df_ex['positiveRate'].fillna(0)
df_ex['testsPerCase'] = df_ex['testsPerCase'].fillna(0)
df_ex['newVaccinations'] = df_ex['newVaccinations'].fillna(0)

In [ ]:
df_ex.dtypes

In [ ]:
# Cambio tipo de dato a entero
df_ex['icuPatients'] = df_ex['icuPatients'].astype(int)
df_ex['hospPatients'] = df_ex['hospPatients'].astype(int)
df_ex['newVaccinations'] = df_ex['newVaccinations'].astype(int)

In [ ]:
# Control de calidad de nulos
print(df_ex['icuPatients'].isnull().sum())
print(df_ex['hospPatients'].isnull().sum())
print(df_ex['positiveRate'].isnull().sum())
print(df_ex['testsPerCase'].isnull().sum())
print(df_ex['newVaccinations'].isnull().sum())

In [ ]:
df_ex.dtypes

Control de Calidad ok

##### 5.1.5.2. Columnas `totalTests`, `vaccinatedPerHundred` y `fullyVaccinatedPerHundred`

Al ser valores acumulativo continuos por dia, los registros nulos debemos de rellenarlos con el registro anterior puesto que no tenemos mas informacion. 
Los Registros del primer dia en este momento son nulos. Rellenando ese dia con 0 despues podemos rellenarlos con el valor anterior, ya que en caso de que no haya datos, por lo mneos podemos decir que nop habran cambiado y no sera del todo incorrecto ya que son valores que van escalando con el paso del tiempo.

In [ ]:
# Con esta nueva columnas sabremos cuando cambia de pais a nivel de fila. Ya que al ser el registo de country diferente al anterior, match sera igual 0 
df_ex['match'] = df_ex.country == df_ex.country.shift()

df_ex['match'].dtypes

In [ ]:
# Cambio tipo de dato a match de boll a str
df_ex['match'] = df_ex['match'].astype(str)

# Replace de bool a 1 y 0
df_ex['match'] = df_ex['match'].replace('False', '0')
df_ex['match'] = df_ex['match'].replace('True', '1')


In [ ]:
# Cuando match sea 0 el primer registro de un pais es 0. De esta forma puedo hacer un df.fillna(method='pad') sin que afecte a otros paises ya que el primer registro es 0
df_ex.loc[df_ex.match == '0', 'totalTests'] = '0'
df_ex.loc[df_ex.match == '0', 'vaccinatedPerHundred'] = '0'
df_ex.loc[df_ex.match == '0', 'fullyVaccinatedPerHundred'] = '0'
df_ex.head(2)

In [ ]:
df_ex['totalTests'] = df_ex['totalTests'].fillna(method='pad')
df_ex['vaccinatedPerHundred'] = df_ex['vaccinatedPerHundred'].fillna(method='pad')
df_ex['fullyVaccinatedPerHundred'] = df_ex['fullyVaccinatedPerHundred'].fillna(method='pad')
df_ex.head(5)

In [ ]:
df_ex.dtypes

In [ ]:
df_DD = df_ex.drop_duplicates()

print(f"Filas df: {df_ex.shape[0]}\nFilas df sin duplicados: {df_DD.shape[0]}")
n_duplicados = df_ex.shape[0] - df_DD.shape[0]
print(f"Hay {n_duplicados} filas duplicadas")
n_null = df_ex.isnull().sum().sum()
print(f"Hay {n_null} registros nulos en total")
print(df_ex.isnull().sum())

In [ ]:
df_ex['totalTests'] = df_ex['totalTests'].astype(int)
df_ex['vaccinatedPerHundred'] = df_ex['vaccinatedPerHundred'].astype(float)
df_ex['fullyVaccinatedPerHundred'] = df_ex['fullyVaccinatedPerHundred'].astype(float)

##### 5.1.5.3. Columna `population`

In [ ]:
df_ex[df_ex['population'].isnull() == True]

In [ ]:
df_ex[df_ex['population'].isnull() == True]['country'].unique()

Vemos que los nulos correspondes a paises que no interesan para el analisis, ya que no se encuentran en `df`. Por lo tanto los eliminaremos

In [ ]:
df_ex =  df_ex.dropna()

In [ ]:
df_DD = df_ex.drop_duplicates()

print(f"Filas df: {df_ex.shape[0]}\nFilas df sin duplicados: {df_DD.shape[0]}")
n_duplicados = df_ex.shape[0] - df_DD.shape[0]
print(f"Hay {n_duplicados} filas duplicadas")
n_null = df_ex.isnull().sum().sum()
print(f"Hay {n_null} registros nulos en total")
print(df_ex.isnull().sum())

Control de calidad ok

In [ ]:
# Cambio population a int
df_ex['population'] = df_ex['population'].astype(int)

#### 5.1.6. Creando `id` para mergear con `df`

In [ ]:
# Defino la columna que me serviran para mergear con otros dataset
df_ex['id-merge'] = df_ex['country'] + df_ex['date'].apply(str)
df_ex['id-merge'] = df_ex['id-merge'].apply(lambda x: x.split(' ')[0])
df_ex['id-merge']

In [ ]:
# Elimino columnas duplicados con df
df_ex = df_ex.drop(['country', 'date', 'match'], axis=1)


Control de calidad ok. 
Dataframe listo para mergear

## 6. Enriquecimiento de `df`

Realizamos inner porque no me interesan registros que esten en un df y en otro no. Son dos datasets que se van actualizando diariamente pero puedo hacer diferencias entre paises en los ultimos dias. pero no resulta sigficativo para dicha tarea.

In [ ]:
# Mergeo con el dataset de test
df = pd.merge(df, df_ex, how='inner', on='id-merge')
df

In [ ]:
df_DD = df.drop_duplicates()

print(f"Filas df: {df.shape[0]}\nFilas df sin duplicados: {df_DD.shape[0]}")
n_duplicados = df.shape[0] - df_DD.shape[0]
print(f"Hay {n_duplicados} filas duplicadas")
n_null = df.isnull().sum().sum()
print(f"Hay {n_null} registros nulos en total")
print(df.isnull().sum())

In [ ]:
print(df.dtypes)

In [ ]:
print(len(df['country'].unique()))


In [ ]:
date_min = df['date'].min()
date_max = df['date'].max()
dias = date_max - date_min
print(date_min,date_max, dias)

## 7. Exportacion `df` to `.csv`

In [ ]:
df.to_csv('df.csv')